In [ ]:
#Ethel Cherotaw 
#AIM2 Week0 Task_2 
#8/23/2024

import pandas as pad
import matplotlib.pyplot as plo
import seaborn as sns
import numpy as num
from scipy import stats
from scipy.stats import zscore
import streamlit as st
dt_ben = pad.read_csv('notebooks/sierraleone-bumbuna.csv') # to read file # dt_ben: data 
dt_ben.describe() # to display few data set

In [ ]:
# Summary statistics to find Measure of center,Spread,IQR: mean, meadian and mode ..etc
# it is possible to use the df.descibe() 
stat_summ = dt_ben.describe()
stat_summ

In [ ]:
# check for data quality , missing values , outliers....

missing_values = dt_ben.isnull().sum()
print("Missing values  column:\n",missing_values)
#Negetive value check on "GHI","DDNI","DHI"
negative_values_ghi = dt_ben[dt_ben['GHI'] < 0].shape[0]
negative_values_dni = dt_ben[dt_ben['DNI'] < 0].shape[0]
negative_values_dhi = dt_ben[dt_ben['DHI'] < 0].shape[0]
print(f"Negative values in GHI: {negative_values_ghi}")
print(f"Negative values in DNI: {negative_values_dni}")
print(f"Negative values in DHI: {negative_values_dhi}")
#Checking Outliers 
z_scores = num.abs(stats.zscore(dt_ben[['ModA', 'ModB', 'WS', 'WSgust']]))

# Outliers are defined as points where Z-score > 3
outliers = num.where(z_scores > 3)
print("Outliers detected at positions (row, column):\n", outliers)

# Count the number of outliers
outliers_count = (z_scores > 3).sum(axis=0)
print("Number of outliers in each column:\n", outliers_count)

In [ ]:
# Time Series Analysis
plo.figure(figsize=(15, 8))

# Plotting GHI, DNI, DHI
plo.subplot(2, 1, 1)
plo.plot(dt_ben.index, dt_ben['GHI'], label='GHI', color='orange', alpha=0.7)
plo.plot(dt_ben.index, dt_ben['DNI'], label='DNI', color='red', alpha=0.7)
plo.plot(dt_ben.index, dt_ben['DHI'], label='DHI', color='blue', alpha=0.7)
plo.title('Time Series of Solar Irradiance Components (GHI, DNI, DHI) - benin-malanville')
plo.xlabel('Time')
plo.ylabel('Irradiance (W/m²)')
# Specify legend location to avoid performance issues
plo.legend(loc='upper right')

# Plotting Tamb (Ambient Temperature)
plo.subplot(2, 1, 2)
plo.plot(dt_ben.index, dt_ben['Tamb'], label='Tamb (Ambient Temperature)', color='green', alpha=0.7)
plo.title('Time Series of Ambient Temperature (Tamb) - benin-malanville')
plo.xlabel('Time')
plo.ylabel('Temperature (°C)')
# Specify legend location to avoid performance issues
plo.legend(loc='upper right')

plo.tight_layout()
plo.show()

In [ ]:
# Evaluating the impact of data: cleaning the coulumn on the sensor readings (ModA, ModB) 
dt_ben = pad.read_csv('benin-malanville.csv')
dt_ben['Timestamp'] = pad.to_datetime(dt_ben['Timestamp'])
#  'Timestamp' as the index
dt_ben.set_index('Timestamp', inplace=True)
dt_ben['IsCleaned'] = dt_ben['Cleaning'] == 1  # Assuming '1' indicates cleaning
# data before and after cleaning datasets
before_cleaning = dt_ben[dt_ben['IsCleaned'] == False][['ModA', 'ModB']]
after_cleaning = dt_ben[dt_ben['IsCleaned'] == True][['ModA', 'ModB']]
# Dtect outlier as we did before using Z-score ,
# IQR can be used based on data distribution (symetric or non symetric)
def detect_outliers_z_score(data, threshold=3):
    # Calculate Z-scores
    z_scores = (data - data.mean()) / data.std()
    # Identify outliers
    is_outlier = (num.abs(z_scores) > threshold)
    return is_outlier.sum()
# Number of outliers before and after cleaning using Z-score
outliers_before_cleaning = before_cleaning.apply(detect_outliers_z_score)
outliers_after_cleaning = after_cleaning.apply(detect_outliers_z_score)
print("\nOutliers before cleaning (Z-score):\n", outliers_before_cleaning)
print("\nOutliers after cleaning (Z-score):\n", outliers_after_cleaning)
# Remove outliers based on Z-score
def remove_outliers_z_score(data, threshold=3):
    z_scores = (data - data.mean()) / data.std()
    # Remove outliers
    filtered_data = data[~(num.abs(z_scores) > threshold).any(axis=1)]
    return filtered_data
before_cleaning_cleaned = remove_outliers_z_score(before_cleaning)
after_cleaning_cleaned = remove_outliers_z_score(after_cleaning)
#Summary statistics after outlier removal
before_stats_cleaned = before_cleaning_cleaned.describe()
after_stats_cleaned = after_cleaning_cleaned.describe()
print("\nSummary statistics before cleaning (after outlier removal):\n", before_stats_cleaned)
print("\nSummary statistics after cleaning (after outlier removal):\n", after_stats_cleaned)
# Visualize the cleaned data (after removing outliers)
plo.figure(figsize=(15, 6))
plo.plot(before_cleaning_cleaned.index, before_cleaning_cleaned['ModA'], label='ModA (Before Cleaning)', color='blue', alpha=0.7)
plo.plot(before_cleaning_cleaned.index, before_cleaning_cleaned['ModB'], label='ModB (Before Cleaning)', color='green', alpha=0.7)
plo.plot(after_cleaning_cleaned.index, after_cleaning_cleaned['ModA'], label='ModA (After Cleaning)', color='cyan', alpha=0.7)
plo.plot(after_cleaning_cleaned.index, after_cleaning_cleaned['ModB'], label='ModB (After Cleaning)', color='orange', alpha=0.7)

plo.title('Sensor Readings (ModA and ModB) Over Time After Outlier Removal')
plo.xlabel('Time')
plo.ylabel('Sensor Reading')
plo.legend()
plo.show()

# Correlation analysis 
print("Columns in dataset:", dt_ben.columns)
print("First few rows of the dataset:\n", dt_ben.head())

# Define relevant columns
solar_radiation_columns = ['GHI', 'DNI', 'DHI']
temperature_columns = ['TModA', 'TModB']
wind_columns = ['WS', 'WSgust', 'WD']
relevant_columns = solar_radiation_columns + temperature_columns + wind_columns
data = dt_ben[relevant_columns]
data_clean = data.dropna()

# Correlation matrix for heatmap
correlation_matrix = data_clean.corr()

# Plot heatmap to visualize correlations between solar radiation and temperature measures
plo.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plo.title('Correlation Heatmap of Solar Radiation and Temperature Measures')
plo.show()

# Scatter matrix to investigate relationships between wind conditions and solar irradiance
sns.pairplot(data_clean, vars=wind_columns + solar_radiation_columns, diag_kind='kde')
plo.suptitle('Scatter Matrix of Wind Conditions and Solar Irradiance', y=1.02)
plo.show()

In [ ]:
# Wind Analysis: Use Polar plots 
wind_direction = dt_ben['WD']     
wind_speed = dt_ben['WS']        
wind_direction_rad = num.deg2rad(wind_direction)
plo.figure(figsize=(12, 6))
ax1 = plo.subplot(121, projection='polar')
sc = ax1.scatter(wind_direction_rad, wind_speed, c=wind_speed, cmap='viridis', s=10, alpha=0.75, edgecolors='w')
ax1.set_title('Wind Speed Distribution by Direction')
plo.colorbar(sc, ax=ax1, label='Wind Speed (m/s)')
hist, bins = num.histogram(wind_direction_rad, bins=36, range=(0, 2 * num.pi))
width = num.diff(bins)
ax2 = plo.subplot(122, projection='polar')
bars = ax2.bar(bins[:-1], hist, width=width, align='center', alpha=0.7, color='orange', edgecolor='black')
ax2.set_title('Wind Direction Variability')
plo.tight_layout()
plo.show()

In [ ]:
#print("Columns in the cleaned DataFrame:", dt_ben.columns)

# Calculate the correlation matrix
corr_matrix = dt_ben[['RH', 'GHI', 'DNI', 'DHI', 'Tamb', 'TModA', 'TModB']].corr()

# Plot the correlation matrix
plo.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plo.title('Correlation Matrix: RH, Solar Radiation, and Temperature-benin-malanville')
plo.show()

# Scatter plots to visualize relationships between RH and other variables
plo.figure(figsize=(15, 5))

plo.subplot(1, 3, 1)
sns.scatterplot(x='RH', y='GHI', data=dt_ben, alpha=0.3)
plo.title('RH vs GHI')

plo.subplot(1, 3, 2)
sns.scatterplot(x='RH', y='Tamb', data=dt_ben, alpha=0.3)
plo.title('RH vs Tamb')

plo.subplot(1, 3, 3)
sns.scatterplot(x='RH', y='TModA', data=dt_ben, alpha=0.3)
plo.title('RH vs TModA')

plo.tight_layout()
plo.show()



In [ ]:
plo.figure(figsize=(15, 12))
variables = {
    'GHI': 'Global Horizontal Irradiance (GHI)',
    'DNI': 'Direct Normal Irradiance (DNI)',
    'DHI': 'Diffuse Horizontal Irradiance (DHI)',
    'WS': 'Wind Speed (WS)',
    'Tamb': 'Ambient Temperature (Tamb)',
    'TModA': 'Temperature of Module A (TModA)',
    'TModB': 'Temperature of Module B (TModB)'
}
# Create subplots for each variable
for i, (var, title) in enumerate(variables.items(), start=1):
    plo.subplot(3, 3, i)  # Adjust subplot dimensions if needed
    plo.hist(dt_ben[var].dropna(), bins=30, edgecolor='black', alpha=0.7)
    plo.title(title)
    plo.xlabel('Value')
    plo.ylabel('Frequency')

# Adjust layout
plo.tight_layout()
plo.show()

In [ ]:
# Z-SCORE ANALYSIS
def calculate_z_scores(df):
    z_scores = pad.DataFrame(index=df.index)
    
    for column in df.columns:
        if df[column].dtype in [num.float64, num.int64]:  
            
            mean = df[column].mean()
            std_dev = df[column].std()
            # Calculate Z-scores
            z_scores[column] = (df[column] - mean) / std_dev
            df[f'{column}_outlier'] = (num.abs(z_scores[column]) > 3).astype(int)
    
    return z_scores, df
# Calculate Z-scores and flag outliers for all numeric columns
z_scores, dt_ben_with_outliers = calculate_z_scores(dt_ben)
# Display Z-scores and flagged outliers
print("Z-Scores:")
print(z_scores.head())
print("\nDataset with Outliers Flagged:")
print(dt_ben_with_outliers.head())

In [ ]:
sample_data = dt_ben[['GHI', 'Tamb', 'RH', 'WS']].dropna()

# Create Bubble Chart
plo.figure(figsize=(10, 6))
scatter = plo.scatter(
    sample_data['GHI'], 
    sample_data['Tamb'], 
    s=sample_data['RH'] * 10,  # Bubble size, scaled by 10 for visibility
    alpha=0.5, 
    c=sample_data['WS'],  # Bubble color represents WS
    cmap='viridis', 
    edgecolors='w'
)

# Add labels and title
plo.xlabel('GHI (W/m²)')
plo.ylabel('Tamb (°C)')
plo.title('Bubble Chart: GHI vs Tamb with Bubble Size Representing RH and Color Representing WS')

# Add colorbar
cbar = plo.colorbar(scatter)
cbar.set_label('WS (m/s)')

# Show plot
plo.show()

